# Youtube Ranking

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver.exe')   #'chromedriver' 만 써도 됨
driver.get(url)
time.sleep(2)

- Step 1,2

In [3]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

- Step 3

In [4]:
tr = trs[0]
# 순위
tr.select_one('.rank').get_text().strip()


'1'

In [5]:
# 카테고리
#tr.select_one('.category').get_text().strip()
tr.select_one('.category').get_text().strip()[1:-1]             # [] 제거

'음악/댄스/가수'

In [6]:
# 채널명
tr.select_one('.subject a').get_text().strip()
# 구독자
tr.select_one('.subscriber_cnt').get_text().strip()
# 조회수
tr.select_one('.view_cnt').get_text().strip()
# 비디오 갯수
#tr.select_one('.video_cnt').get_text().strip()
tr.select_one('.video_cnt').get_text().strip()[:-1]                  # 마지막에 '개' 빼줌

'394'

- Step 4

In [7]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject a').get_text().strip()
    subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
    view = tr.select_one('.view_cnt').get_text().strip()
    video = tr.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category, channel, subscriber, view, video])

- Step 5

In [8]:
df = pd.DataFrame(lines, columns=['순위','카테고리','채널','구독자수','조회수','비디오수'])
df.head(3)

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,7350만,233억1663만,394개
1,2,음악/댄스/가수,BANGTANTV,6550만,157억9463만,"1,686개"
2,3,음악/댄스/가수,HYBE LABELS,6500만,221억9069만,818개


In [9]:
df.tail(3)

,순위,카테고리,채널,구독자수,조회수,비디오수
97,98,미분류,MayTree,410만,7억5549만,173개
98,99,취미/라이프,JaeYeol ASMR 재열,409만,11억6453만,"1,085개"
99,100,TV/방송,채널 십오야,404만,8억8278만,547개


- 숫자로 바꿔주는 함수 (단위가 여러개라 lambda 사용 어려움)

In [10]:
def convert_unit(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return f'{int(s):,d}'

In [11]:
convert_unit('233억1663만')

'23,316,630,000'

In [12]:
for column in df.columns[3:]:                           # 3부터 끝까지
    df[column] = df[column].apply(convert_unit)         # apply 뒤에 람다 뿐만아니라 함수이름도 사용가능
df.head()

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"73,500,000","23,316,630,000",394
1,2,음악/댄스/가수,BANGTANTV,"65,500,000","15,794,630,000","1,686"
2,3,음악/댄스/가수,HYBE LABELS,"65,000,000","22,190,690,000",818
3,4,음악/댄스/가수,SMTOWN,"30,300,000","24,310,770,000","3,895"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


- Step 6: 두번째 페이지

- Step 6: 모든 페이지

In [13]:
# Xpath 로 찾아가기 - //*[@id="list-skin"]/nav/span/a[1]
# 페이지명 찾아서 우클릭, copy - Xpath
#driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()
#time.sleep(2)

In [14]:
from tqdm.notebook import tqdm

In [15]:
lines = []

for page in tqdm(range(1,11)):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject a').get_text().strip()
        subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
        view = tr.select_one('.view_cnt').get_text().strip()
        video = tr.select_one('.video_cnt').get_text().strip()
        lines.append([rank, category, channel, subscriber, view, video])

  0%|          | 0/10 [00:00<?, ?it/s]

- Step 7 

In [16]:
driver.close()

In [20]:
df = pd.DataFrame(lines, columns=['순위','카테고리','채널','구독자수','조회수','비디오수'])
for column in df.columns[3:]:                           
    df[column] = df[column].apply(convert_unit)         
df

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"73,500,000","23,316,630,000",394
1,2,음악/댄스/가수,BANGTANTV,"65,500,000","15,794,630,000","1,686"
2,3,음악/댄스/가수,HYBE LABELS,"65,000,000","22,190,690,000",818
3,4,음악/댄스/가수,SMTOWN,"30,300,000","24,310,770,000","3,895"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
...,...,...,...,...,...,...
995,994,애완/반려동물,김메주와 고양이들,"580,000","209,780,000","1,060"
996,997,미분류,일주어터,"580,000","118,380,000",194
997,1001,미분류,쏘대장,"580,000","131,650,000",352
998,1000,영화/만화/애니,네이버 웹툰,"580,000","432,110,000",833


In [21]:
df.to_csv('Youtube랭킹.csv', index=False)